Upload "Mental_Health_FAQ.csv" dan "stopwords.txt" Sebelum Run Code

# **Import Library**

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import string

# **Read File**

In [ ]:
df = pd.read_csv('Mental_Health_FAQ.csv')

In [ ]:
df.drop('Question_ID', axis=1, inplace=True)

# **Split**

In [ ]:
question = df['Questions'].tolist()
answer = df['Answers'].tolist()

# **LowerCase**

In [ ]:
question = [q.lower() for q in question]
answer = [a.lower() for a in answer]

# **StopWords**

In [ ]:
stopwords = []
with open('stopwords.txt', encoding='utf-8') as f:
  for line in f:
    stopwords.append(line.strip())

# **Cleaning**

In [ ]:
question_clean = []
for i in question:
  temp = [w for w in i.split() if w not in stopwords]
  temp = " ".join(temp)
  temp = re.sub(r'[.,’"\'-?:!;]', '', temp)
  temp = re.sub(r'^whats|^im', '', temp)
  question_clean.append(temp.strip())

In [ ]:
answer_clean = []
for i in answer:
  temp = [w for w in i.split()]
  temp = " ".join(temp)
  temp = temp.encode("ascii", "ignore")
  answer_clean.append(temp.decode())

# **Tokenizing & Padding**

In [ ]:
def fit_tokenizer(train_sentences, num_words, oov_token, max_len):
    
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)

    
    tokenizer.fit_on_texts(train_sentences)
    sequences = tokenizer.texts_to_sequences(train_sentences)
    padd_seq = pad_sequences(sequences, truncating='post', maxlen=max_len)
    
    return padd_seq, tokenizer

In [ ]:
train_seq, tokenizer = fit_tokenizer(question_clean, 1000, "<OOV>", 100)

In [ ]:
lb = LabelEncoder()
val_label_seq = lb.fit_transform(answer_clean)


# **Modelling**

In [ ]:
class Callback(tf.keras.callbacks.Callback):
    # Define the method that checks the accuracy at the end of each epoch
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.90:
            print("\n\nAccuracy Has Been Achieved Around 90%, Training Stop!\n")
            self.model.stop_training = True

In [ ]:
def create_model(vocab_size, embedding_dim, maxlen):
    
    model = tf.keras.Sequential([ 
        tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=maxlen),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(98, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']) 


    return model

In [ ]:
model = create_model(len(tokenizer.word_index), 16, 100)
history = model.fit(train_seq, val_label_seq, epochs=100, callbacks=[Callback()])

Epoch 1/100
4/4 [==============================] - 0s 5ms/step - loss: 4.5880 - accuracy: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 5ms/step - loss: 4.5823 - accuracy: 0.0408
Epoch 3/100
4/4 [==============================] - 0s 6ms/step - loss: 4.5813 - accuracy: 0.0204
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 4.5783 - accuracy: 0.0510
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 4.5757 - accuracy: 0.0612
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 4.5725 - accuracy: 0.0612
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 4.5691 - accuracy: 0.0408
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 4.5655 - accuracy: 0.0204
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 4.5610 - accuracy: 0.0306
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 4.5584 - accuracy: 0.0306
Epoch 11/100
4/4 [=======

# **Inference**

In [ ]:
while(True):  
  user_input = input("Input Sentence : ")
  if user_input == 'exit':
    break

  user_input = [w for w in user_input.split() if w not in stopwords]
  user_input = " ".join(user_input)
  user_input = re.sub(r'[.,’"\'-?:!;]', '', user_input)
  user_input = re.sub(r'^whats|^im', '', user_input)
  user_input = user_input.strip()

  result = model.predict(pad_sequences(tokenizer.texts_to_sequences([user_input]),
                                              truncating='post', maxlen=100))

  bot_reply = lb.inverse_transform([np.argmax(result)])
  print(f"Bot : {bot_reply[0]}")

Input Sentence : what does mental illness mean ?
Bot : mental illnesses are health conditions that disrupt a persons thoughts, emotions, relationships, and daily functioning. they are associated with distress and diminished capacity to engage in the ordinary activities of daily life. mental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. on the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. these include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital. it is important to know that mental illnesses are medical conditions that have nothing to do with a persons character, intelligence, or willpower. just as diabetes is a disorder of the pancreas, mental illness is a medical condition due to the brains biology. similarly to how one 

# **Save Model**

In [ ]:
# model.save("capstone_model")

In [ ]:
# import pickle
# with open('tokenizer.pickle', 'wb') as handle:
#     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('label_encoder.pickle', 'wb') as f:
#     pickle.dump(lb, f, protocol=pickle.HIGHEST_PROTOCOL)\
# with open('model.pkl', 'wb') as f:
#     pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)